In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "ENZYMES"

# *Baseline*

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)

def run_seed(seed):
    set_seed(seed)
    
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name)

    # Filter out None graphs and create x if missing
    clean_dataset = []
    for data in dataset:
        if data is None:
            continue
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        clean_dataset.append(data)

    if len(clean_dataset) == 0:
        raise RuntimeError(f"No valid graphs in dataset. Check your {Dataset_name} installation.")

    dataset = clean_dataset

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.shape[1]
    out_channels = max([data.y.max().item() for data in dataset]) + 1
    model = GCN(in_channels, 64, out_channels).to(device)

    # Shuffle using random.shuffle instead of dataset.shuffle()
    random.shuffle(dataset)

    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_baseline_seed_{seed}",
        group=f"GCN_{Dataset_name}_baseline",
        job_type="baseline",
        reinit=True,
        config={
            "model":"GCN",
            "dataset":Dataset_name,
            "hidden_channels":64,
            "lr":0.001,
            "epochs":EPOCHS,
            "task":"Baseline",
            "seed":seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, 31):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_085527-4r6t8mon
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_baseline_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/4r6t8mon


Seed 1 | Epoch 001 | Loss: 1.7972 | Test Acc: 0.2000
Seed 1 | Epoch 002 | Loss: 1.7886 | Test Acc: 0.2167
Seed 1 | Epoch 003 | Loss: 1.7833 | Test Acc: 0.1500
Seed 1 | Epoch 004 | Loss: 1.7773 | Test Acc: 0.1833
Seed 1 | Epoch 005 | Loss: 1.7704 | Test Acc: 0.1750
Seed 1 | Epoch 006 | Loss: 1.7643 | Test Acc: 0.1833
Seed 1 | Epoch 007 | Loss: 1.7597 | Test Acc: 0.1833
Seed 1 | Epoch 008 | Loss: 1.7536 | Test Acc: 0.1667
Seed 1 | Epoch 009 | Loss: 1.7514 | Test Acc: 0.1917
Seed 1 | Epoch 010 | Loss: 1.7473 | Test Acc: 0.1833
Seed 1 | Epoch 011 | Loss: 1.7456 | Test Acc: 0.1667
Seed 1 | Epoch 012 | Loss: 1.7447 | Test Acc: 0.1833
Seed 1 | Epoch 013 | Loss: 1.7422 | Test Acc: 0.2083
Seed 1 | Epoch 014 | Loss: 1.7419 | Test Acc: 0.1833
Seed 1 | Epoch 015 | Loss: 1.7394 | Test Acc: 0.1583
Seed 1 | Epoch 016 | Loss: 1.7372 | Test Acc: 0.1667
Seed 1 | Epoch 017 | Loss: 1.7366 | Test Acc: 0.2000
Seed 1 | Epoch 018 | Loss: 1.7334 | Test Acc: 0.2000
Seed 1 | Epoch 019 | Loss: 1.7331 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb:  test_acc ▅▆▁▄▃▄▂▄▄▅▂▂▅▅▄▅▅▆▃▄▅▅▅▇▄▆▆▅▇▇▇▅▇▅▅▇▇▇█▅
wandb: train_acc ▁▆▄▇▇▆▅▆▆▆▆▆▆▆▇▅▇█▇█▇███▇█▇▇██▇▇▇▇▇█▇█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.69826
wandb:  test_acc 0.2
wandb: train_acc 0.29792
wandb: 
wandb: 🚀 View run GCN_baseline_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/4r6t8mon
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_085527-4r6t8mon/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/r

Seed 2 | Epoch 001 | Loss: 1.7917 | Test Acc: 0.0917
Seed 2 | Epoch 002 | Loss: 1.7846 | Test Acc: 0.0917
Seed 2 | Epoch 003 | Loss: 1.7797 | Test Acc: 0.0833
Seed 2 | Epoch 004 | Loss: 1.7750 | Test Acc: 0.1417
Seed 2 | Epoch 005 | Loss: 1.7695 | Test Acc: 0.1500
Seed 2 | Epoch 006 | Loss: 1.7636 | Test Acc: 0.1417
Seed 2 | Epoch 007 | Loss: 1.7572 | Test Acc: 0.1417
Seed 2 | Epoch 008 | Loss: 1.7515 | Test Acc: 0.1583
Seed 2 | Epoch 009 | Loss: 1.7450 | Test Acc: 0.1500
Seed 2 | Epoch 010 | Loss: 1.7427 | Test Acc: 0.1417
Seed 2 | Epoch 011 | Loss: 1.7381 | Test Acc: 0.1500
Seed 2 | Epoch 012 | Loss: 1.7387 | Test Acc: 0.1833
Seed 2 | Epoch 013 | Loss: 1.7366 | Test Acc: 0.1333
Seed 2 | Epoch 014 | Loss: 1.7319 | Test Acc: 0.2000
Seed 2 | Epoch 015 | Loss: 1.7275 | Test Acc: 0.1667
Seed 2 | Epoch 016 | Loss: 1.7265 | Test Acc: 0.1750
Seed 2 | Epoch 017 | Loss: 1.7230 | Test Acc: 0.1583
Seed 2 | Epoch 018 | Loss: 1.7194 | Test Acc: 0.1833
Seed 2 | Epoch 019 | Loss: 1.7178 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁
wandb:  test_acc ▁▁▄▅▄▅▄▅▆▄▅▆▅▆▅▆▇▆█▆▇▇▇▇█▇█▇█▆█▆█▆▆▇█▇▇█
wandb: train_acc ▁▁▁▄▄▃▃▄▅▅▄▇▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇▇▇▇▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.68364
wandb:  test_acc 0.20833
wandb: train_acc 0.29167
wandb: 
wandb: 🚀 View run GCN_baseline_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/vehk4aus
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_085543-vehk4aus/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wan

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁
wandb:  test_acc ▁▁▁▂▂▄▅▅▃▄▅▅▄▅▆▆▅▆▅▅▆▆▇▇▇▇▇▇▇▇▇█▇█▇█▇█▇▇
wandb: train_acc ▁▁▁▁▂▃▃▄▅▅▆▆▅▇▇▇▇▇▇▇▇▆█▇██████▇█▇███▇█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.69928
wandb:  test_acc 0.24167
wandb: train_acc 0.28958
wandb: 
wandb: 🚀 View run GCN_baseline_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/j61l7rav
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_090147-j61l7rav/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wa

Seed 29 | Epoch 001 | Loss: 1.7935 | Test Acc: 0.1333
Seed 29 | Epoch 002 | Loss: 1.7850 | Test Acc: 0.1083
Seed 29 | Epoch 003 | Loss: 1.7792 | Test Acc: 0.1250
Seed 29 | Epoch 004 | Loss: 1.7742 | Test Acc: 0.1583
Seed 29 | Epoch 005 | Loss: 1.7669 | Test Acc: 0.1667
Seed 29 | Epoch 006 | Loss: 1.7617 | Test Acc: 0.1833
Seed 29 | Epoch 007 | Loss: 1.7574 | Test Acc: 0.1667
Seed 29 | Epoch 008 | Loss: 1.7546 | Test Acc: 0.1500
Seed 29 | Epoch 009 | Loss: 1.7497 | Test Acc: 0.1500
Seed 29 | Epoch 010 | Loss: 1.7483 | Test Acc: 0.1917
Seed 29 | Epoch 011 | Loss: 1.7451 | Test Acc: 0.1917
Seed 29 | Epoch 012 | Loss: 1.7430 | Test Acc: 0.1917
Seed 29 | Epoch 013 | Loss: 1.7419 | Test Acc: 0.1833
Seed 29 | Epoch 014 | Loss: 1.7402 | Test Acc: 0.1750
Seed 29 | Epoch 015 | Loss: 1.7387 | Test Acc: 0.2167
Seed 29 | Epoch 016 | Loss: 1.7367 | Test Acc: 0.2333
Seed 29 | Epoch 017 | Loss: 1.7348 | Test Acc: 0.2417
Seed 29 | Epoch 018 | Loss: 1.7327 | Test Acc: 0.1917
Seed 29 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss █▇▇▇▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁
wandb:  test_acc ▂▁▂▄▅▃▃▅▅▅▅▇██▅█▆█▆▇▇██▇▆█▇▆▇▆▇▆▆█▇█▇█▆▆
wandb: train_acc ▁▂▂▄▃▃▃▂▄▄▄▅▅▅▅▅▅▇▆▆▇▇▇▆▇█▅█▇▇▆▇▇▇▇▇▇█▇▆
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.69136
wandb:  test_acc 0.2
wandb: train_acc 0.28958
wandb: 
wandb: 🚀 View run GCN_baseline_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/dcrf8wm5
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_090200-dcrf8wm5/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/

Seed 30 | Epoch 001 | Loss: 1.7977 | Test Acc: 0.1750
Seed 30 | Epoch 002 | Loss: 1.7893 | Test Acc: 0.1833
Seed 30 | Epoch 003 | Loss: 1.7854 | Test Acc: 0.1417
Seed 30 | Epoch 004 | Loss: 1.7816 | Test Acc: 0.1417
Seed 30 | Epoch 005 | Loss: 1.7773 | Test Acc: 0.1500
Seed 30 | Epoch 006 | Loss: 1.7731 | Test Acc: 0.1500
Seed 30 | Epoch 007 | Loss: 1.7681 | Test Acc: 0.1667
Seed 30 | Epoch 008 | Loss: 1.7626 | Test Acc: 0.1333
Seed 30 | Epoch 009 | Loss: 1.7585 | Test Acc: 0.1417
Seed 30 | Epoch 010 | Loss: 1.7554 | Test Acc: 0.1750
Seed 30 | Epoch 011 | Loss: 1.7519 | Test Acc: 0.1750
Seed 30 | Epoch 012 | Loss: 1.7493 | Test Acc: 0.2083
Seed 30 | Epoch 013 | Loss: 1.7473 | Test Acc: 0.1667
Seed 30 | Epoch 014 | Loss: 1.7474 | Test Acc: 0.1583
Seed 30 | Epoch 015 | Loss: 1.7458 | Test Acc: 0.1917
Seed 30 | Epoch 016 | Loss: 1.7411 | Test Acc: 0.1500
Seed 30 | Epoch 017 | Loss: 1.7398 | Test Acc: 0.1750
Seed 30 | Epoch 018 | Loss: 1.7390 | Test Acc: 0.2083
Seed 30 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
wandb:  test_acc ▄▄▂▂▂▁▂▄▄▆▃▅▂▄▆▄█▇▅▅▄▆▇▅▄▇▅▆▄▇▇▇▄▅▇▆▇▆▆▇
wandb: train_acc ▁▆▅▆▄▄▅▅▅▅▅▄▆▆▅▅▅▆▇▅▇██▆▆█▆▆█▇▇▆█▇█▇▇▇▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.6978
wandb:  test_acc 0.225
wandb: train_acc 0.28542
wandb: 
wandb: 🚀 View run GCN_baseline_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/zkjv6j65
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_090214-zkjv6j65/logs


# *BFS*

In [4]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def add_bfs_rank(d):
    d = d.clone()
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    G = to_networkx(d, to_undirected=True)
    root = int(d.edge_index[0][0].item()) if d.edge_index.numel() > 0 else 0
    bfs_order = list(nx.bfs_tree(G, root)) if G.number_of_nodes() > 0 else []
    bfs_rank = torch.full((d.num_nodes, 1), -1.0, dtype=torch.float)
    L = max(1, len(bfs_order))
    for i, node in enumerate(bfs_order):
        bfs_rank[node] = float(i) / float(L)
    d.x = torch.cat([d.x, bfs_rank], dim=1)
    return d

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=add_bfs_rank)
    torch.manual_seed(seed)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = GCN(in_channels, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_BFS_seed_{seed}",
        group=f"GCN_{Dataset_name}_BFS",
        job_type="bfs",
        reinit=True,
        config={"model":"GCN","hidden_channels":64,"epochs":EPOCHS,"lr":0.001,"dataset":"REDDIT-BINARY","task":"BFS","positional_encoding":"BFS","seed":seed}
    )

    def train():
        model.train()
        total_loss = 0.0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_090229-jdchyr4a
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_BFS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/jdchyr4a


Seed 1 | Epoch 001 | Loss: 1.7954 | Test Acc: 0.1667
Seed 1 | Epoch 002 | Loss: 1.7897 | Test Acc: 0.1750
Seed 1 | Epoch 003 | Loss: 1.7843 | Test Acc: 0.1917
Seed 1 | Epoch 004 | Loss: 1.7813 | Test Acc: 0.1667
Seed 1 | Epoch 005 | Loss: 1.7778 | Test Acc: 0.1917
Seed 1 | Epoch 006 | Loss: 1.7736 | Test Acc: 0.1667
Seed 1 | Epoch 007 | Loss: 1.7692 | Test Acc: 0.1750
Seed 1 | Epoch 008 | Loss: 1.7650 | Test Acc: 0.1833
Seed 1 | Epoch 009 | Loss: 1.7612 | Test Acc: 0.2000
Seed 1 | Epoch 010 | Loss: 1.7565 | Test Acc: 0.2167
Seed 1 | Epoch 011 | Loss: 1.7522 | Test Acc: 0.1917
Seed 1 | Epoch 012 | Loss: 1.7492 | Test Acc: 0.1917
Seed 1 | Epoch 013 | Loss: 1.7461 | Test Acc: 0.2250
Seed 1 | Epoch 014 | Loss: 1.7456 | Test Acc: 0.2333
Seed 1 | Epoch 015 | Loss: 1.7386 | Test Acc: 0.2083
Seed 1 | Epoch 016 | Loss: 1.7395 | Test Acc: 0.2917
Seed 1 | Epoch 017 | Loss: 1.7316 | Test Acc: 0.2500
Seed 1 | Epoch 018 | Loss: 1.7294 | Test Acc: 0.2500
Seed 1 | Epoch 019 | Loss: 1.7275 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇▇▇▆▆▆▆▆▅▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
wandb:  test_acc ▁▂▁▂▁▂▃▃▂▂▄▃▇▅▅▇▅▇▆▆█▆▆▇█▆▆▇▇▅█▆██▇▇▆▇▇▇
wandb: train_acc ▁▂▅▄▄▄▅▅▅▄▄▅▆▅▆▆▆▇▇▆▆▇▆▇▇█▇▇█▆▇▇██▇██▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.6374
wandb:  test_acc 0.28333
wandb: train_acc 0.32083
wandb: 
wandb: 🚀 View run GCN_BFS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/jdchyr4a
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_090229-jdchyr4a/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run

Seed 2 | Epoch 001 | Loss: 1.7966 | Test Acc: 0.1667
Seed 2 | Epoch 002 | Loss: 1.7863 | Test Acc: 0.2250
Seed 2 | Epoch 003 | Loss: 1.7824 | Test Acc: 0.2250
Seed 2 | Epoch 004 | Loss: 1.7770 | Test Acc: 0.2083
Seed 2 | Epoch 005 | Loss: 1.7720 | Test Acc: 0.2000
Seed 2 | Epoch 006 | Loss: 1.7644 | Test Acc: 0.2167
Seed 2 | Epoch 007 | Loss: 1.7602 | Test Acc: 0.2167
Seed 2 | Epoch 008 | Loss: 1.7523 | Test Acc: 0.2250
Seed 2 | Epoch 009 | Loss: 1.7471 | Test Acc: 0.2167
Seed 2 | Epoch 010 | Loss: 1.7419 | Test Acc: 0.2083
Seed 2 | Epoch 011 | Loss: 1.7389 | Test Acc: 0.2333
Seed 2 | Epoch 012 | Loss: 1.7360 | Test Acc: 0.2250
Seed 2 | Epoch 013 | Loss: 1.7328 | Test Acc: 0.2167
Seed 2 | Epoch 014 | Loss: 1.7301 | Test Acc: 0.2083
Seed 2 | Epoch 015 | Loss: 1.7298 | Test Acc: 0.2250
Seed 2 | Epoch 016 | Loss: 1.7272 | Test Acc: 0.2417
Seed 2 | Epoch 017 | Loss: 1.7221 | Test Acc: 0.2333
Seed 2 | Epoch 018 | Loss: 1.7187 | Test Acc: 0.2083
Seed 2 | Epoch 019 | Loss: 1.7168 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
wandb:  test_acc ▁▄▄▃▂▄▃▃▄▄▃▄▄▄▃▄▄▄▄▄▄▅▅▅▅▅▆▅▅▅▅▇▇▇▆▆▇▇█▇
wandb: train_acc ▁▃▄▄▄▃▄▄▃▄▄▄▆▄▅▆▆▅▇▆▆█▆▇▆▇█▇██▇▇█▇▇▇█▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.64784
wandb:  test_acc 0.30833
wandb: train_acc 0.30417
wandb: 
wandb: 🚀 View run GCN_BFS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/8p2o2xoy
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_090338-8p2o2xoy/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/ru

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
wandb:  test_acc ▁▂▂▂▂▄▄▄▄▄▃▆█▅▄▆▇▇▅▄▄▅▅▆▇▇▇▅▇█▆▇▇▇█▇█▇▅▇
wandb: train_acc ▁▃▃▅▄▄▄▄▄▅▅▅▆▆▅▇▆▆▇▆▇▆▆▆▇▆▇▇▇███▇████▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.62933
wandb:  test_acc 0.23333
wandb: train_acc 0.30833
wandb: 
wandb: 🚀 View run GCN_BFS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/bvhv7lq4
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_093413-bvhv7lq4/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/r

Seed 29 | Epoch 001 | Loss: 1.7948 | Test Acc: 0.1417
Seed 29 | Epoch 002 | Loss: 1.7868 | Test Acc: 0.1500
Seed 29 | Epoch 003 | Loss: 1.7828 | Test Acc: 0.2000
Seed 29 | Epoch 004 | Loss: 1.7769 | Test Acc: 0.2167
Seed 29 | Epoch 005 | Loss: 1.7723 | Test Acc: 0.2167
Seed 29 | Epoch 006 | Loss: 1.7668 | Test Acc: 0.2083
Seed 29 | Epoch 007 | Loss: 1.7607 | Test Acc: 0.2083
Seed 29 | Epoch 008 | Loss: 1.7550 | Test Acc: 0.2083
Seed 29 | Epoch 009 | Loss: 1.7487 | Test Acc: 0.2000
Seed 29 | Epoch 010 | Loss: 1.7462 | Test Acc: 0.2167
Seed 29 | Epoch 011 | Loss: 1.7409 | Test Acc: 0.2250
Seed 29 | Epoch 012 | Loss: 1.7361 | Test Acc: 0.2333
Seed 29 | Epoch 013 | Loss: 1.7307 | Test Acc: 0.2250
Seed 29 | Epoch 014 | Loss: 1.7268 | Test Acc: 0.2500
Seed 29 | Epoch 015 | Loss: 1.7227 | Test Acc: 0.2583
Seed 29 | Epoch 016 | Loss: 1.7190 | Test Acc: 0.2500
Seed 29 | Epoch 017 | Loss: 1.7174 | Test Acc: 0.2667
Seed 29 | Epoch 018 | Loss: 1.7127 | Test Acc: 0.2583
Seed 29 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▁▁
wandb:  test_acc ▁▁▃▄▄▄▃▄▄▅▅▆▅▆▆▆▆▇▇▇▇█▇▆▇█▆▆▇▇▇▇▇▇▇▇▇▇▇▇
wandb: train_acc ▁▂▄▅▅▄▅▅▄▅▆▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇▇▇▇▇▇█▇▇▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.62014
wandb:  test_acc 0.28333
wandb: train_acc 0.3
wandb: 
wandb: 🚀 View run GCN_BFS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/z7ya2k14
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_093523-z7ya2k14/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-2

Seed 30 | Epoch 001 | Loss: 1.7985 | Test Acc: 0.1667
Seed 30 | Epoch 002 | Loss: 1.7889 | Test Acc: 0.1333
Seed 30 | Epoch 003 | Loss: 1.7835 | Test Acc: 0.1333
Seed 30 | Epoch 004 | Loss: 1.7793 | Test Acc: 0.1667
Seed 30 | Epoch 005 | Loss: 1.7758 | Test Acc: 0.1583
Seed 30 | Epoch 006 | Loss: 1.7709 | Test Acc: 0.2000
Seed 30 | Epoch 007 | Loss: 1.7669 | Test Acc: 0.2000
Seed 30 | Epoch 008 | Loss: 1.7624 | Test Acc: 0.2333
Seed 30 | Epoch 009 | Loss: 1.7592 | Test Acc: 0.2500
Seed 30 | Epoch 010 | Loss: 1.7547 | Test Acc: 0.2417
Seed 30 | Epoch 011 | Loss: 1.7507 | Test Acc: 0.2250
Seed 30 | Epoch 012 | Loss: 1.7482 | Test Acc: 0.2167
Seed 30 | Epoch 013 | Loss: 1.7485 | Test Acc: 0.2583
Seed 30 | Epoch 014 | Loss: 1.7412 | Test Acc: 0.2333
Seed 30 | Epoch 015 | Loss: 1.7394 | Test Acc: 0.2500
Seed 30 | Epoch 016 | Loss: 1.7370 | Test Acc: 0.2250
Seed 30 | Epoch 017 | Loss: 1.7351 | Test Acc: 0.2167
Seed 30 | Epoch 018 | Loss: 1.7327 | Test Acc: 0.2417
Seed 30 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
wandb:  test_acc ▃▁▁▃▂▄▆▆▆▅▇▆▆▅▅▆▅▅▆▆▆▅▆▅▆▆▆▅▅▆▆▇█▇▆▇▇▇▇▇
wandb: train_acc ▁▃▃▁▄▄▃▃▄▄▄▄▄▄▄▃▄▅▄▅▅▅▆▆▅▆▆▆▇▇▇▇▇▇██▇███
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.65241
wandb:  test_acc 0.26667
wandb: train_acc 0.32917
wandb: 
wandb: 🚀 View run GCN_BFS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/k5x8nbfk
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_093633-k5x8nbfk/logs
